In [1]:
from DrissionPage import ChromiumPage, ChromiumOptions
import pandas as pd
import time
import random  # 新增随机模块
import html_to_json
import json
from datetime import datetime

co = ChromiumOptions()
# co.headless(True)  # 取消注释可启用无头模式

page = ChromiumPage(co)
code = input('请输入股票代码：')

start_date = input('请输入开始日期（格式：YYYY-MM-DD）：')

end_date = input('请输入结束日期（格式：YYYY-MM-DD）：')


# 将输入的日期字符串转换为datetime对象
start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')

page.get(f'https://guba.eastmoney.com/list,{code}_1.html')
nums = int(page.ele('xpath:/html/body/div[1]/div[3]/div[1]/div[1]/div/ul/li[1]/ul/li[7]/a/span').text)
print(f"总页数: {nums}")

# 用于存储爬取的数据
data = {
    '标题': [],
    '作者': [],
    '阅读数': [],
    '评论数': [],
    '最后更新时间': []
}

# 用于标记是否继续爬取（当帖子日期早于结束日期时停止）
continue_crawling = True

for i in range(1, nums + 1):
    if not continue_crawling:
        break

    print(f"正在爬取第 {i} 页...")
    page.get(f'https://guba.eastmoney.com/list,{code}_{i}.html')
    
    # 新增：页面加载后随机延时1-3秒，模拟人类操作节奏
    delay = random.uniform(1, 5)
    print(f"页面加载完成，等待 {delay:.2f} 秒...")
    time.sleep(delay)

    html_content = page.html

    try:
        json_data = \
        html_to_json.convert(html_content)['html'][0]['body'][0]['script'][0]['_value'][17:].split('var')[0][:-5]

        json_data = json.loads(json_data)

        for j in json_data['re']:
            # 提取帖子的最后更新日期
            post_date_str = j['post_last_time'].split(' ')[0]
            post_date_obj = datetime.strptime(post_date_str, '%Y-%m-%d')

            # 判断帖子日期是否在指定范围内
            if start_date_obj <= post_date_obj <= end_date_obj:

                # 将数据添加到字典中
                data['标题'].append(j['post_title'])
                if 'post_user' in j:
                    data['作者'].append(j['post_user']['user_nickname'])
                else:
                    data['作者'].append(j['user_nickname'])
                data['阅读数'].append(j['post_click_count'])
                data['评论数'].append(j['post_comment_count'])
                data['最后更新时间'].append(j['post_last_time'])

                # 打印数据（可选）
                print(f"标题: {j['post_title']}")
                # print(f"作者: {j['user_nickname']}")
                print(f"阅读数: {j['post_click_count']}")
                print(f"评论数: {j['post_comment_count']}")
                print(f"最后更新时间: {j['post_last_time']}")
                print("-" * 50)
            elif post_date_obj < end_date_obj:
                # 如果帖子日期早于结束日期，停止爬取后续页面
                continue_crawling = False
                break

    except Exception as e:
        print(f"处理第 {i} 页时出错: {e}")
        continue

# 将数据转换为DataFrame
df = pd.DataFrame(data)

# 保存到Excel文件
if not df.empty:
    filename = f"{code}_{start_date}_{end_date}_股吧数据.xlsx"
    df.to_excel(filename, index=False, engine='openpyxl')
    print(f"数据已成功保存到 {filename}")
else:
    print("没有找到符合条件的数据，未生成Excel文件。")

print("爬取完成！")
    

总页数: 30
正在爬取第 1 页...
页面加载完成，等待 3.82 秒...
标题: 明天A股涨。
阅读数: 1085
评论数: 0
最后更新时间: 2023-01-05 02:56:18
--------------------------------------------------
标题: 明天A股涨。！
阅读数: 1020
评论数: 0
最后更新时间: 2023-01-05 02:55:41
--------------------------------------------------
标题: 美股下跌过程中三个界限分明的走势段剖析（思辨三角形整理和背离整理）
阅读数: 534
评论数: 2
最后更新时间: 2022-11-04 16:49:19
--------------------------------------------------
标题: UVXY------逐渐被压缩到极致的弹簧（区间套理论在趋势性走势中的实战应用）
阅读数: 1105
评论数: 1
最后更新时间: 2022-06-24 17:06:51
--------------------------------------------------
标题: 美股的再次熔断开启时间剖析（大致时间将在下下周/即2022年6月底）
阅读数: 993
评论数: 0
最后更新时间: 2022-06-19 17:00:56
--------------------------------------------------
标题: 【巴菲特：不会停止在标普500指数上的投资标普500ETF（513500）成交额
阅读数: 1122
评论数: 0
最后更新时间: 2022-05-05 15:54:12
--------------------------------------------------
标题: 巴菲特股东大会将在本周六重磅来袭。现年92岁的巴菲特和98岁的芒格又将回归线下，
阅读数: 1094
评论数: 0
最后更新时间: 2022-04-29 17:53:28
--------------------------------------------------
标题: 【标普500ETF（513500）飘红Meta一季度盈利74亿美元】美股三大股指
阅读数: 1201

In [8]:
import pandas as pd
from snownlp import SnowNLP

target_list=[['PVC']]
for i in target_list:
    df1=pd.read_excel('data/'+i[0]+'_股吧数据_2020-2024.xlsx')
    df1['score'] = df1['标题'].apply(lambda x: SnowNLP(str(x)).sentiments)
    df1['emotion'] = df1['score'].apply(lambda x: 'positive' if x > 0.5 else 'negative')
    df2=df1[['标题','最后更新时间','阅读数','score','emotion']]
    print(df2)
df2.to_excel('data/打分/'+i[0]+'_打分.xlsx',index=0)


                                              标题               最后更新时间  阅读数  \
0                                 5300的本，诶，回本有望把  2024-12-31 23:36:08  223   
1       原油那边爆涨，pvc这边大跌，几年难见的大行情可能要来了，成本端一涨，啧啧，这些  2024-12-31 22:26:13  191   
2                                    这玩意就这么不值钱吗？  2024-12-31 22:11:24  142   
3                                今天收大碌柱，今晚低开直接梭哈  2024-12-31 18:06:06  190   
4               多单可以持仓过节，说不定为了恢复经济，在元旦晚会上释放重棒利好。  2024-12-31 17:46:58  210   
...                                          ...                  ...  ...   
150264                                  这玩应做的人少？  2020-01-03 11:16:31  287   
150265                                     周线看涨！  2020-01-02 16:33:24  469   
150266                                橡胶拿着拿住空单继续  2020-01-02 14:45:08  270   
150267                               拉尾盘你们信么[大笑]  2020-01-02 14:31:58  353   
150268                            所有都是高开低走，除了沥青。  2020-01-02 09:11:58  366   

           score   emotion  
0       0.070468  negative  
1    

## 清除格式差

In [ ]:
import pandas as pd

target_list=[['PVC'],['沪铜'],['玉米']]

for i in target_list:
    df2 = pd.read_csv('data/'+i[0]+'.csv')
    df = pd.read_excel('data/打分/'+i[0]+'_打分.xlsx')
    
    # df['最后更新时间'] = df['最后更新时间'].apply(lambda x: '2023年' + x if len(x) < 14 else x)
    df['最后更新时间'] = pd.to_datetime(df['最后更新时间'], format='%Y年%m月%d日%H:%M')
    df['最后更新时间'] = df['最后更新时间'].dt.strftime('%Y-%m-%d')
    df_sorted = df.sort_values('最后更新时间')
    df_sorted[['最后更新时间','标题','score']]
    df_sorted['emotion'] = df_sorted['score'].apply(lambda x: 'positive' if x > 0.5 else 'negative')
    df_sorted = df_sorted.sort_values('最后更新时间')
    average_scores = df_sorted.groupby('最后更新时间')['score'].mean().reset_index()
    average_scores.rename(columns={'最后更新时间': 'time'}, inplace=True)
    
    # 将average_scores中的'日期'列转换为datetime64[ns]类型，和df2中的对应列类型保持一致（假设df2中是该类型）
    average_scores['time'] = pd.to_datetime(average_scores['time'])
    
    merged_df = pd.merge(average_scores, df2, on='time')
    merged_df[['time','open', 'high' ,'low' ,'close', 'volume', 'amount','score']]
    merged_df.to_csv('data/pycharm用/'+i[0]+'数据+得分.csv',index=0)